<a href="https://colab.research.google.com/github/mustafabozkaya/DeepLearningBootcamp2022/blob/master/Bootcamp_Solution/Spectogram_Recognition_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img align="Left" width="100" height="100" src="https://thumbs.dreamstime.com/b/mb-initial-letter-vector-logo-icon-mb-initial-letter-vector-logo-icon-204517753.jpg">](https://github.com/mustafabozkaya)

# Spectogram Recognition with CNN

---
[<img align="Left" width="800" height="300" src="https://www.researchgate.net/publication/319081627/figure/fig1/AS:534034566004736@1504335170521/Spectrogram-of-a-speech-signal-with-breath-sound-marked-as-Breath-whose-bounds-are.png">](#)



**Colab** için kimlik doğrulama adımları:

**Drive yükleme işlemi**

**Drive da dosya konumlandırmayı yapma işlemleri**

In [1]:
from google.colab import drive
drive.mount('/content/mydrive/',force_remount=True)

MessageError: ignored

In [ ]:
#unmount drive
# drive.flush_and_unmount()

In [ ]:
!pwd


In [ ]:
%cd /content

In [ ]:
!pwd

In [ ]:
#Copying current content to new editable directory
#!cp -r "../content/drive/MyDrive/spectrograms/" "/sample_data/"


+
# Package İnstalling and Controlling

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
!cat /proc/meminfo

In [ ]:
!pip install -q keras
!pip install -q Pillow

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()


In [ ]:
tf.__version__

In [ ]:
#https://github.com/astrada/google-drive-ocamlfuse/
# !mkdir -p drive
# !google-drive-ocamlfuse drive
# !ls

In [ ]:
!ls ../content

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras. layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import glob
import numpy as np
import os
import shutil
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"]=(18,8)
%matplotlib inline
np.random.seed(42)

In [ ]:
cv.__version__

# Configuring İmage datasets folder 

In [ ]:
dataset_dir=f"/content/mydrive/My Drive/spectrograms/"


In [ ]:
for root, dirs, files in os.walk(dataset_dir, topdown=False):
    print(root)
    print(f"files type :{type(files)}")
    print(f"files lenth :{len(files)}")
    #print(os.path.join(root, name))
    print(f"DİR type :{type(dirs)}")
    print(f"DİR lenth :{len(dirs)}")
    
    

In [ ]:
for root,dirname, filenames in os.walk(dataset_dir):
    for filename in (filenames):
        print(os.path.join(root, filename))
        break
    

In [ ]:
!ls

# Data Exploration

In [ ]:

class_name="1" # set to class name

train_files = glob.glob(str(os.path.join(dataset_dir,str(class_name)))+'/*')

print(*train_files)
print(f"1 spectogram img lenth :{len(train_files)}")
train_imgs = [img_to_array(load_img(img)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = np.array([fn.split('/')[5].split(".")[0].strip() for fn in train_files]) # target , y label

print('Train dataset shape:', train_imgs.shape)
print('Train labels shape:', train_labels.shape)

In [ ]:
train_imgs[100]

In [ ]:
train_labels[100]

In [ ]:
# create image datasets 

In [ ]:
image_size=(374,500)
batch_size=32 # paketler

#Setting train/test split
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    
    directory=dataset_dir,
    labels="inferred",
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=image_size,
    batch_size=batch_size,
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_dir,
    labels="inferred",
    validation_split=0.2,
    subset="validation",
    seed=1007,
    image_size=image_size,
    batch_size=batch_size,
)
#Checking images and labels shapes (amount of images, height, width, color channels)
for image_batch, labels_batch in test_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
train_ds.class_names

In [ ]:
print(type(train_ds))

In [ ]:
for i in train_ds.take(5):
  print(i[0].shape)

In [ ]:
#Checking images and labels shapes (amount of images, height, width, color channels)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

# Data Visualizaiton

In [ ]:
#Displaying image samples 
plt.figure(figsize=(18, 18))
for images, labels in train_ds.take(1):
    for i in range(batch_size):
        ax = plt.subplot(4,8,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

# Data Preprocessing

Before we can build our model and start training, we need to apply one simple augmentation the dataset and that is rescaling. We rescale an input in the (0, 255) range to be in the (0,1) range.

In [ ]:
# Function to prepare our datasets for modelling
def prepare(ds, augment=False):
    # Define our one transformation
    rescale = tf.keras.Sequential(
        [tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
    flip_and_rotate = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip(
            "horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    ])

    # Apply rescale to both datasets and augmentation only to training
    ds = ds.map(lambda x, y: (rescale(x, training=True), y))
    if augment:
        ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y))
    return ds


train_dataset = prepare(train_ds, augment=False)
valid_dataset = prepare(test_ds, augment=False)


# Modelling

In [ ]:
#Checking if the data format i.e the RGB channel is coming first or last so, whatever it may be, model will check first and then input shape will be feeded accordingly.
from keras import backend as K
if K.image_data_format() == "channels_first":
    input_shape = (3, img_height, img_width)
else:
    input_shape = (img_height, img_width, 3)


In [ ]:
def conv_net(X_train, y_train): 

    model = tf.keras.Sequential()

    # Create CNN model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
    model.add(tf.keras.layers.Conv2D(
        32, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

  
    
    return model

In [ ]:
model_dl=conv_net()

# Model Visualization

In [ ]:
model_dl.summary()


In [ ]:

# Compile model
model_dl.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=['accuracy'],
)

### 

In [ ]:

# Train model for 10 epochs, capture the history
history = model_dl.fit(train_dataset,batch_size=batch_size, epochs=10, validation_data=valid_dataset)


In [ ]:
model_dl.save('save_models/spectogram_model.h5')

**Rastgele değer için test işlemi**

In [ ]:
model_test = model_dl.save('save_models/spectogram_model.h5')

# Model Evaluation

In [ ]:
# Plot the loss curves for training and validation.
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values)+1)

plt.figure(figsize=(8, 6))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Plot the accuracy curves for training and validation.
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(acc_values)+1)

plt.figure(figsize=(8, 6))
plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


We can compute the final loss and accuracy score on our valid dataset using the evaluate() function.

In [ ]:
# Compute the final loss and accuracy
final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(
    final_loss, final_acc))
